# Person-wise Photo Segmentation & Organization System

In [10]:
import cv2
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn import svm
import numpy as np
import os
import face_recognition
import pickle
import shutil

In [4]:
for person in os.listdir("Face Recongition Model"):
    person_path = os.path.join("Face Recongition Model",person)
    for image in os.listdir(person_path):
        image_path = os.path.join(person_path , image)
        print(image_path)

Face Recongition Model\10915602722_Payal Arora\.ipynb_checkpoints
Face Recongition Model\10915602722_Payal Arora\Payal Arora_10915602722_1.jpg
Face Recongition Model\10915602722_Payal Arora\Payal Arora_10915602722_10.jpg
Face Recongition Model\10915602722_Payal Arora\Payal Arora_10915602722_11.jpg
Face Recongition Model\10915602722_Payal Arora\Payal Arora_10915602722_12.jpg
Face Recongition Model\10915602722_Payal Arora\Payal Arora_10915602722_13.jpg
Face Recongition Model\10915602722_Payal Arora\Payal Arora_10915602722_14.jpg
Face Recongition Model\10915602722_Payal Arora\Payal Arora_10915602722_15.jpg
Face Recongition Model\10915602722_Payal Arora\Payal Arora_10915602722_16.jpg
Face Recongition Model\10915602722_Payal Arora\Payal Arora_10915602722_17.jpg
Face Recongition Model\10915602722_Payal Arora\Payal Arora_10915602722_18.jpg
Face Recongition Model\10915602722_Payal Arora\Payal Arora_10915602722_19.jpg
Face Recongition Model\10915602722_Payal Arora\Payal Arora_10915602722_20.jpg

In [6]:
dataset = "Face Recongition Model"
encodings = []
labels = []

for person in os.listdir(dataset):
    person_path = os.path.join(dataset,person)
    if not person_path:
        continue
    for image_path in os.listdir(person_path):
        image = os.path.join(person_path , image_path)
        img = cv2.imread(image)
        if img is None:
            continue
        rgb_image = cv2.cvtColor(img , cv2.COLOR_BGR2RGB)
        locations = face_recognition.face_locations(rgb_image)
        if len(locations)==0:
            continue
        face_encodings = face_recognition.face_encodings(rgb_image , locations)
        for encoding in face_encodings:
            encodings.append(encoding)
            labels.append(person)


label = LabelEncoder()
y = label.fit_transform(labels)

svc = svm.SVC(kernel='linear',probability=True)
svc.fit(encodings , y)

with open("svc_clf_18july.pkl","wb") as f:
    pickle.dump(svc,f)

In [7]:
with open("labels_18july.pkl","wb") as f:
    pickle.dump(label ,f)

# Single Face detection in an image

In [11]:
def preprocess_image(img_path):
    image = face_recognition.load_image_file(img_path)
    face_location = face_recognition.face_locations(image)
    if not face_location:
        return None
    face_encoding = face_recognition.face_encodings(image , face_location)
    if face_encoding:
        return face_encoding[0]
    return None
        

In [13]:
with open("labels_18july.pkl","rb") as f:
    label = pickle.load(f)

with open("svc_clf_18july.pkl","rb") as f:
    model = pickle.load(f)

test = "Gallery"
output = "Sorted_Gallery"

os.makedirs(output,exist_ok=True)

for person in os.listdir(test):
    image_path = os.path.join(test , person)
    img = preprocess_image(image_path)
    if img is not None:
        prediction = model.predict([img])[0]
        name = label.inverse_transform([prediction])[0]
        path = os.path.join(output,name)
        os.makedirs(path , exist_ok=True)
        shutil.copy(image_path , os.path.join(path,person))
        print(f"{person} - > {name}")
    else:
        print(f"🚫 No face found in {person}, skipped.")
    

IMG-20250718-WA0030.jpg - > 20315602722_KARTIK ARORA
IMG-20250718-WA0031.jpg - > 20315602722_KARTIK ARORA
IMG-20250718-WA0032.jpg - > 20315602722_KARTIK ARORA
IMG-20250718-WA0033.jpg - > 20315602722_KARTIK ARORA
IMG-20250718-WA0034.jpg - > 10915602722_Payal Arora
IMG-20250718-WA0035.jpg - > 10915602722_Payal Arora
IMG-20250718-WA0036.jpg - > 11015602722_Suman Arora
IMG-20250718-WA0037.jpg - > 10915602722_Payal Arora
IMG-20250718-WA0038.jpg - > 10915602722_Payal Arora
IMG-20250718-WA0039.jpg - > 10915602722_Payal Arora
IMG-20250718-WA0040.jpg - > 11015602722_Suman Arora
IMG-20250718-WA0041.jpg - > 11015602722_Suman Arora
IMG-20250718-WA0042.jpg - > 11015602722_Suman Arora
IMG-20250718-WA0043.jpg - > 10915602722_Payal Arora
IMG-20250718-WA0044.jpg - > 11115602722_Shivam Arora
IMG-20250718-WA0045.jpg - > 11115602722_Shivam Arora
IMG-20250718-WA0046.jpg - > 15615602722_Rajeev Kumar
IMG-20250718-WA0047.jpg - > 20315602722_KARTIK ARORA
IMG-20250718-WA0048.jpg - > 10915602722_Payal Arora
IMG-

# Returning Multiple Faces 

# Person-wise Photo Segmentation & Organization System

In [14]:
def preprocess_multiple(img_path):
    image = face_recognition.load_image_file(img_path)
    face_locations = face_recognition.face_locations(image)
    if not face_locations:
        return None
    face_encodings = face_recognition.face_encodings(image , face_locations)
    return face_encodings

In [17]:
with open("svc_clf_18july.pkl","rb") as f:
    model = pickle.load(f)

with open("labels_18july.pkl","rb") as f:
    label = pickle.load(f)


output_folder = "Sorted_Gallery_Multiple_Faces_Detection"
test = "Gallery"

for img_file in os.listdir(test):
    image_path = os.path.join(test , img_file)
    encodings = preprocess_multiple(image_path)
    if not encodings:
        print(f"🚫 No faces found in {img_file}, skipped.")
        continue

    person_detected = set()
    for encoding in encodings:
        prediction = model.predict([encoding])[0]
        person_name = label.inverse_transform([prediction])[0]
        person_detected.add(person_name)

    for person in person_detected:
        person_folder = os.path.join(output_folder , person)
        os.makedirs(person_folder , exist_ok = True)

        dest_path = os.path.join(person_folder , img_file)
        shutil.copy(image_path , dest_path)
        
    print(f"✅ {img_file} → {', '.join(person_detected)}")


✅ IMG-20250718-WA0030.jpg → 20315602722_KARTIK ARORA
✅ IMG-20250718-WA0031.jpg → 20315602722_KARTIK ARORA
✅ IMG-20250718-WA0032.jpg → 20315602722_KARTIK ARORA
✅ IMG-20250718-WA0033.jpg → 20315602722_KARTIK ARORA
✅ IMG-20250718-WA0034.jpg → 10915602722_Payal Arora
✅ IMG-20250718-WA0035.jpg → 10915602722_Payal Arora
✅ IMG-20250718-WA0036.jpg → 10915602722_Payal Arora, 11015602722_Suman Arora
✅ IMG-20250718-WA0037.jpg → 10915602722_Payal Arora
✅ IMG-20250718-WA0038.jpg → 10915602722_Payal Arora
✅ IMG-20250718-WA0039.jpg → 10915602722_Payal Arora, 15615602722_Rajeev Kumar
✅ IMG-20250718-WA0040.jpg → 11015602722_Suman Arora
✅ IMG-20250718-WA0041.jpg → 11015602722_Suman Arora
✅ IMG-20250718-WA0042.jpg → 11015602722_Suman Arora
✅ IMG-20250718-WA0043.jpg → 10915602722_Payal Arora, 11115602722_Shivam Arora
✅ IMG-20250718-WA0044.jpg → 10915602722_Payal Arora, 11115602722_Shivam Arora
✅ IMG-20250718-WA0045.jpg → 11115602722_Shivam Arora
✅ IMG-20250718-WA0046.jpg → 15615602722_Rajeev Kumar, 111156